In [1]:
from netgen.occ import *

order = 3
r = 50
l = 500
s = 10
dl = l / s
maxh = 50

shapes = []
for i in range(s):
    v = i * dl
    arc1 = ArcOfCircle(Pnt(v,0,-2*r), Pnt(v,-r,-r), Pnt(v,0,0))
    arc2 = ArcOfCircle(Pnt(v,0,0), Pnt(v,r,r), Pnt(v,0,2*r))
    circ1 = arc1.Extrude( dl * X )
    circ1.edges.Min(Z).name = "hor"
    circ1.edges.Max(Z).name = "mid"
    circ1.edges.Max(X).name = "vert"

    flat1 = circ1.edges.Max(Z).Extrude( 100 * Y )
    flat2 = circ1.edges.Max(Z).Extrude( -100 * Y )
    flat1.edges.Max(Y).name = "flat"
    flat2.edges.Min(Y).name = "flat"
    shapes.append(flat1)
    shapes.append(flat2)
    
    circ2 = arc2.Extrude( dl * X )
    circ2.edges.Max(Z).name = "hor"
    circ2.edges.Max(X).name = "vert"

    if i == 0:
        circ1.edges.Min(X).name = "back"
        circ2.edges.Min(X).name = "back"
        flat1.edges.Min(X).name = "frame"
        flat2.edges.Min(X).name = "frame"

    if i == 9:
        flat1.edges.Max(X).name = "frame"
        flat2.edges.Max(X).name = "frame"
    
    cyli = circ1+circ2
    shapes.append(cyli)


cyl = Glue(shapes)

geo = OCCGeometry(cyl)

from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh(geo.GenerateMesh(maxh=maxh)) 
mesh.Curve(order)

Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [5]:
n = specialcf.normal(3)

print(mesh.GetNE(BND))

Draw(n[1], mesh)
Draw(n[2], mesh)

620
Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |  
 |  Parameters:
 |  
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |  
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |  
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |  
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [3]:
t = specialcf.tangential(3)  
Draw(t, mesh.BBoundaries("mid|hor|vert|flat|back|frame"))

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [4]:
e1 = CF ( (1,0,0), dims = (3,1) )
e2 = CF ( (0,1,0), dims = (3,1) )
e3 = CF ( (0,0,1), dims = (3,1) )

In [5]:
fesH1 = H1(mesh, order=order, dirichlet_bbnd="back")
fes = fesH1 * fesH1 * fesH1 * fesH1 * fesH1 * fesH1

(u1,u2,u3,r1,r2,r3), (du1,du2,du3,dr1,dr2,dr3) = fes.TnT()

u = CF( (u1,u2,u3), dims = (3,1) )
du = CF( (du1,du2,du3), dims = (3,1) )

r = CF( (r1,r2,r3), dims = (3,1) )
dr = CF( (dr1,dr2,dr3), dims = (3,1) )

In [6]:
q = 1e-4 * CF ( (0,-z, 0), dims = (3,1) )

In [7]:
# Shell projections
def Anti(vec):
    return CF( (0, -vec[2], vec[1],
                vec[2], 0, -vec[0],
                -vec[1], vec[0], 0), dims = (3,3) ) 

def Dyad(v1, v2):
    return CF( (v1[0]*v2[0], v1[0]*v2[1], v1[0]*v2[2],
                v1[1]*v2[0], v1[1]*v2[1], v1[1]*v2[2],
                v1[2]*v2[0], v1[2]*v2[1], v1[2]*v2[2]), dims = (3,3) ) 

Antin = Anti(n) 
Q = Dyad(n, n)
P = Id(3) - Q

# Weingarten tensor
W = -Grad(n)*P

In [8]:
# Beam projections
d2 = CF( (-1, 0, 0), dims = (3,1) )
d3 = Cross( t, d2 )

T = Dyad(t,t)
O = Id(3) - T

Antid2 = Anti(d2)
Antid3 = Anti(d3)

kc = InnerProduct(W,T)

Antie1 = Anti(e1)
Antie2 = Anti(e2)
Antie3 = Anti(e3)

In [9]:
# Shell entities
Dtu = CF ( (grad(u1).Trace(), grad(u2).Trace(), grad(u3).Trace()), dims = (3,3) )  * P
Dtr = CF ( (grad(r1).Trace(), grad(r2).Trace(), grad(r3).Trace()), dims = (3,3) )  * P 

Dtcovu = P * Dtu
Dtcovr = P * Dtr

SymDtcovu = Sym(Dtcovu)  
SkewDtcovu = Skew(Dtcovu)  

R = Anti(r)
Rt = R * P
Rtcov = P * Rt

CurltR = InnerProduct(Dtr, P)*Id(3) - P * Dtr.trans

In [10]:
# Beam entities
Dttu = CF ( (grad(u[0]).Trace().Trace(), grad(u[1]).Trace().Trace(), grad(u[2]).Trace().Trace()), dims = (3,3) )  * T
Dttr = CF ( (grad(r[0]).Trace().Trace(), grad(r[1]).Trace().Trace(), grad(r[2]).Trace().Trace()), dims = (3,3) )  * T  

Dttcovu = T * Dttu
Dttcovr = T * Dttr

divttr = Trace(Dttr)

SymDttcovu = Sym(Dttcovu)  

Rtt = R * T
Rttcov = T * Rt

CurlttR = InnerProduct(Dttr, P)*Id(3) - P * Dttr.trans

In [11]:
# Shell material: silver  
h = 1.6
mues = 30  
lames = 98.5
mucs = 1
Lc = 1e-3

# Plane stress
lames = 2*lames*mues / (lames + 2*mues)


def De(mat):
    return 2*mues*mat + lames*Trace(mat)*P

In [12]:
#  Beam material: graphite
mueb = 2122.64 
lameb = 289.451
mucb = 10000

Ee = mueb * ( 3*lameb + 2 *mueb ) / (lameb + mueb)
A = pi * 0.8**2
I = 0.25 * pi * 0.8**4
    

muelca1 = 10867.9
muelca2 = 122264
muelca3 = 0


def Vol(mat):
    return (1/3) * Trace(mat) * Id(3) 


def Dev(mat):
    return mat - Vol(mat)


def L(mat):
    return  muelca1 * Dev(Sym(mat)) + muelca2 * Skew(mat) + muelca3 * Vol(mat)

In [13]:
a = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=True)

# W = CF ( (0,0,0,
#          0,0,0,
#          0,0,0), dims = (3,3) )

# Shell
a += Variation(0.5*(
    h * InnerProduct(SymDtcovu,De(SymDtcovu)) 
    + h * 2*mucs * InnerProduct(SkewDtcovu - Rtcov, SkewDtcovu - Rtcov)
    + h * (mues + mucs) * InnerProduct(Q*(Dtu - Rt), Q*(Dtu - Rt))
    + (h**3 / 12) * InnerProduct(Sym(Antin * Dtcovr + P*R*W), De(Sym(Antin * Dtcovr + P*R*W)))
    + (h**3 / 12) * 2*mucs * InnerProduct(Skew(Antin * Dtcovr + P*R*W), Skew(Antin * Dtcovr + P*R*W))
    + (h**3 / 12) * (mues + mucs) * InnerProduct(Q*R*W, Q*R*W) 
    + h * mues * Lc**2 * InnerProduct(CurltR, CurltR)
)*ds)

# # Vertical micro-beams
# a += Variation(0.5*(
#     Ee*A * InnerProduct(SymDttcovu, SymDttcovu) 
#     + (mueb + mucb)*A * InnerProduct(O*(Dttr - Rtt), O*(Dttr - Rtt))
#     + A * InnerProduct(CurlttR, L(CurlttR))
# )*dx(mesh.BBoundaries("vert|back")))

# a += Variation(0.5*(
#     Ee*A * InnerProduct(SymDttcovu, SymDttcovu) 
#     + Ee*I * InnerProduct(Sym(P * Antid2 * Dttr), Sym(P * Antid2 * Dttr)) 
#     + Ee*I * InnerProduct(Sym(P * Antid3 * Dttr), Sym(P * Antid3 * Dttr))
#     + (mueb + mucb)*A * InnerProduct(O*(Dttr - Rtt), O*(Dttr - Rtt))
#     + (mueb + mucb)*I * InnerProduct(O*Antid2*Dttr, O*Antid2*Dttr)
#     + (mueb + mucb)*I * InnerProduct(O*(Antid3*Dttr + kc*Rtt), O*(Antid3*Dttr + kc*Rtt))
#     # + (mueb + mucb)*I * InnerProduct(O*(Antid3*Dttr), O*(Antid3*Dttr))
#     + A * InnerProduct(CurlttR, L(CurlttR))
# )*dx(mesh.BBoundaries("vert|back")))

# # Horizontal straight beams
# a += Variation(0.5*(
#     Ee*A * InnerProduct(SymDttcovu, SymDttcovu) 
#     + Ee*I * InnerProduct(Sym(P * Antie2 * Dttr), Sym(P * Antie2 * Dttr)) 
#     + Ee*I * InnerProduct(Sym(P * Antie3 * Dttr), Sym(P * Antie3 * Dttr))
#     + (mueb + mucb)*A * InnerProduct(O*(Dttr - Rtt), O*(Dttr - Rtt))
#     + (mueb + mucb)*I * InnerProduct(O*(Antie2*Dttr + W), O*(Antie2*Dttr + W))
#     + (mueb + mucb)*I * InnerProduct(O*Antie3*Dttr, O*Antie3*Dttr)
#     + A * InnerProduct(CurlttR, L(CurlttR))
# )*dx(mesh.BBoundaries("mid|hor|flat")))

# a += Variation(0.5*(
#     Ee*A * InnerProduct(SymDttcovu, SymDttcovu) 
#     + Ee*I * InnerProduct(Sym(P * Antie1 * Dttr), Sym(P * Antie1 * Dttr)) 
#     + Ee*I * InnerProduct(Sym(P * Antie3 * Dttr), Sym(P * Antie3 * Dttr))
#     + (mueb + mucb)*A * InnerProduct(O*(Dttr - Rtt), O*(Dttr - Rtt))
#     + (mueb + mucb)*I * InnerProduct(O*Antie1*Dttr, O*Antie1*Dttr)
#     + (mueb + mucb)*I * InnerProduct(O*Antie3*Dttr, O*Antie3*Dttr)
#     + A * InnerProduct(CurlttR, L(CurlttR))
# )*dx(mesh.BBoundaries("frame")))

f = LinearForm(fes)
f += InnerProduct(du,q)*dx(mesh.BBoundaries("hor"))

In [14]:
sol = GridFunction(fes)

sol.vec[:] = 0

v = sol.vec.CreateVector()
w = sol.vec.CreateVector()

In [15]:
with TaskManager():
    f.Assemble()
    a.AssembleLinearization(sol.vec)
    inv = a.mat.Inverse(fes.FreeDofs(a.condense), inverse="sparsecholesky")

    a.Apply(sol.vec, v)
    
    v.data -= f.vec
    if a.condense:
        v.data += a.harmonic_extension_trans * v
    w.data = inv * v
    if a.condense:
        w.data += a.harmonic_extension * w
        w.data += a.inner_solve * v
    sol.vec.data -= w

In [16]:
gfu1,gfu2,gfu3, gfr1,gfr2,gfr3 = sol.components

gfu = CF( (gfu1,gfu2,gfu3), dims = (3,1) )
gfr = CF( (gfr1,gfr2,gfr3), dims = (3,1) )
gfDu = CF( (grad(gfu1),grad(gfu2),grad(gfu3)), dims = (3,3) )

Draw(gfu, mesh, "u", deformation=True)
Draw(Norm(gfDu), mesh, "u")
Draw(gfr, mesh, "r")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [19]:
# vtk = VTKOutput(ma=mesh,
#                 coefs=[gfu],
#                 names = ["displacement"],
#                 filename="Sdisp1",
#                 subdivision=3)
# # Exporting the results:
# vtk.Do(vb=BND)

'Sdisp1'